In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
//import scala.io.Source

object Helpers {
    def using[A <: { def close(): Unit }, B](resource: A)(f: A => B): B =
        try {
            f(resource)
        } finally {
            resource.close()
        }
}

import Helpers._

val en : String = "en"
val de : String = "de"
val fr : String = "fr"

val langs : List[String] = List(en, de, fr)

case class Lemma(entry:String, default:String, pos:String)

object Lemma {
  def parse(line:String): Option[Lemma] = {
    line.split(" ") match { 
      case Array(f,s,t) => Some(Lemma(f, s, t))
      case _ => None
    }
  }  
}

val lemmasPerLang: Map[String, List[Lemma]] = Map.empty
val stopwordsPerLang: Map[String, List[String]] = Map.empty

for(lang <- langs) {
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-lemmatizer.txt")) { source => 
    for (line <- source.getLines) {
      val list = lemmasPerLang.getOrElse(lang, List.empty)
      Lemma.parse(line) match { 
        case Some(lemma) => lemmasPerLang.updated(lang, list:+lemma)
        case _ =>;
      }      
    }
  }
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-stopwords.txt")) { source => 
    for (line <- source.getLines) {
      val list = stopwordsPerLang.getOrElse(lang, List.empty)
      stopwordsPerLang.updated(lang, list:+line)  
    }
  }
}





import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
defined object Helpers
import Helpers._
en: String = en
de: String = de
fr: String = fr
langs: List[String] = List(en, de, fr)
defined class Lemma
defined object Lemma
lemmasPerLang: Map[String,List[Lemma]] = Map()
stopwordsPerLang: Map[String,List[String]] = Map()
